In [1]:
# import packages
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace

# set up AzureML workspace
config_path = '../utils/config.json'
tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47'  # this is outputted post `az login`
interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id)  # create log-in object
ws = Workspace.from_config(path=config_path, auth=interactive_auth)  # link workspace


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [16]:
# import packages
import shutil
from sklearn.ensemble import IsolationForest

# copy over script for model creation
shutil.copy('../src/triton_isolation_forest.py', '.')
from triton_isolation_forest import *

# preprocess data
url = 'http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz'  # URL of the public dataset
df = download_data(url)  # read data from the url
df = process_data(df)  # subset the data and select top labels
# split data
train, test_norm, test_anom = split_data(df)
X_train, y_train = train  # unpack training data
# train model
model = IsolationForest(random_state=RAND_STATE)
model.fit(X_train)
# convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType
initial_types = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onx = convert_sklearn(model, initial_types=initial_types)
with open('../src/isolation_forest.onnx', 'wb') as f:
    f.write(onx.SerializeToString())


In [3]:
# import packages
import os

# define helper functions
def instantiate_models(src_file):
    '''
    creates the expected triton directory system given a source model file
    '''
    # extract model name and source file name
    src_name = os.path.basename(src_file)
    model_name = src_name.split('.')[0]  # assumes a <model>.onnx structure
    # set outputted directory name and model filename
    dir_name = f'models/triton/{model_name}/1/'
    file_name = f'{dir_name}{src_name}'
    # copies the data over
    os.makedirs(dir_name, exist_ok=True)  # instantiate directory
    shutil.copy(src_file, file_name)  # copy model onnx file
    # return the file/directory locations
    return dir_name, file_name

def remove_models(src_file):
    '''
    removes the models directory where the triton inputs are stored
    '''
    shutil.rmtree('models')
    os.remove(src_file)

In [4]:
# import packages
from azureml.core.model import Model

# setup model for registration
dir_name,file_name = instantiate_models('../src/isolation_forest.onnx')
# register model
model = Model.register(
    model_path='models',
    model_name='IsolationForestTutorial',
    description='Anomaly detection for KDD network intrusion dataset',
    workspace=ws,
    model_framework=Model.Framework.MULTI,
)


Registering model IsolationForestTutorial


In [5]:
# import packages
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
from azureml.core import Environment

# setup environment (obtain environment.yml from `conda env export > environment.yml`)
env_name,env_path = 'SampleEnv', '../utils/triton_environment.yml'
env = Environment.from_conda_specification(name=env_name, file_path=env_path)

# setup service for deployment
service_name = 'triton-isolation-forest'
infer_config = InferenceConfig('../utils/entry_script.py', environment=env)
deploy_config = LocalWebservice.deploy_configuration(port=6789)
# deploy service
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=infer_config,
    deployment_config=deploy_config,
    overwrite=True,
)
# check deployment status
service.wait_for_deployment(show_output=True)


Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.


Generating Docker build context.
2021/05/20 02:49:42 Downloading source code...
2021/05/20 02:49:44 Finished downloading source code
2021/05/20 02:49:44 Creating Docker network: acb_default_network, driver: 'bridge'
2021/05/20 02:49:45 Successfully set up Docker network: acb_default_network
2021/05/20 02:49:45 Setting up Docker configuration...
2021/05/20 02:49:45 Successfully set up Docker configuration
2021/05/20 02:49:45 Logging in to registry: 27ebdbdbb34a4186a9c6cf92029c3be2.azurecr.io
2021/05/20 02:49:46 Successfully logged into 27ebdbdbb34a4186a9c6cf92029c3be2.azurecr.io
2021/05/20 02:49:46 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/05/20 02:49:46 Scanning for dependencies...
2021/05/20 02:49:47 Successfully scanned dependencies
2021/05/20 02:49:47 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB
Step 1/18 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:2021012

Removing intermediate container c08de400d281
 ---> e5fb0288491c
Step 9/18 : ENV PATH /azureml-envs/azureml_17fe9db7740d671e4d8f2a0e653e1746/bin:$PATH
 ---> Running in 1b2ebab6a6f7
Removing intermediate container 1b2ebab6a6f7
 ---> 03ae4a174522
Step 10/18 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> 8c02bfc7212c
Step 11/18 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> bc6894ad5c60
Step 12/18 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azureml_17fe9db7740d671e4d8f2a0e653e1746
 ---> Running in 43bdbb0d014d
Report materialized dependencies for the environment
Reading environment context
Exporting conda environment
Sending request with materialized conda environment details
Successfully sent materialized environment details
Removing intermediate container 43bdbb0d014d
 ---> 29f17872e19e
Step 13/18 : ENV AZUREML

Step 1/5 : FROM 27ebdbdbb34a4186a9c6cf92029c3be2.azurecr.io/azureml/azureml_6af8a8cb574db902532f0f7595690779
 ---> 4513b502e0dd
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> cbae30bddc3b
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjY1NjA1NzVkLWZhMDYtNGU3ZC05NWZiLWY5NjJlNzRlZmQ3YSIsInJlc291cmNlR3JvdXBOYW1lIjoidXdfYW5vbWFseWRldGVjdGlvbiIsImFjY291bnROYW1lIjoiYW5vbWFseWRldGVjdGlvbiIsIndvcmtzcGFjZUlkIjoiMjdlYmRiZGItYjM0YS00MTg2LWE5YzYtY2Y5MjAyOWMzYmUyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 3a6d8ff7deb4
 ---> 4cb6bfa85fc1
Step 4/5 : RUN mv '/var/azureml-app/tmpqq7b0n1d.py' /var/azureml-app/main.py
 ---> Running in 6628976f63f2
 ---> 345d99346a12
Step 5/5 : CMD ["runsvdir","/var/runit"]
 ---> Running in abba19309078
 ---> b33d2bec6fa3
Successfully built b33d2bec6fa3
Successfully tagged triton-isolation-forest:latest
Container (name:stupefied_johnson, id

In [7]:
print(service.get_logs())


2021-05-20T02:57:25,440833100+00:00 - iot-server/run 
2021-05-20T02:57:25,440757900+00:00 - gunicorn/run 
2021-05-20T02:57:25,447702400+00:00 - rsyslog/run 
2021-05-20T02:57:25,469260300+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-05-20T02:57:25,776992400+00:00 - iot-server/finish 1 0
2021-05-20T02:57:25,778545900+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (13)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 39
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-05-20 02:57:26,133 | root | INFO | Starting up app insights client
2021-05-20 02:57:26,133 | root | INFO | Starting up request id generator
2021-05-20 02:57:26,133 | root | INFO | Starting up app insight hooks
2021-05-20 02:57:26,134 | root | INFO | Invoking user's init function
This is init
2021-05-20 02:57:26,134 | root | INFO | Users's init has 

In [13]:
# clean up
service.delete()
remove_models()


Container has been successfully cleaned up.


In [17]:
remove_models('triton_isolation_forest')

TypeError: remove_models() takes 0 positional arguments but 1 was given